### Link do dataset

In [ ]:
# https://www.kaggle.com/datasets/saisaathvik/house-rent-prices-of-metropolitan-cities-in-india?select=Kolkata_rent.csv

### Descrição dos procedimentos de coleta de dados e significado dos atributos

In [ ]:
# Os dados do dataset foram extraídos através do método Web-Scraping através de uma extensão do Google Chorme, chamada de Instant Data Scraper.
# Link: https://chrome.google.com/webstore/detail/instant-data-scraper/ofaokhiedipichpaobibbnahnkdoiiah

In [ ]:
# O conteúdo do dataset é formado pelos seguintes atributos:
#       * Tipo de vendedor (seller_type);
#       * Numero de Quartos (bedroom);
#       * Tipo do layout (layout_type);
#       * Tipo da propriedade (property_type);
#       * Localização (location);
#       * Preço (price);
#       * Area (area);
#       * Tipo de mobilia (furnish_type);
#       * Numero de Banheiros (bathroom).

In [ ]:
# Dataset ja vem por padrão sem dados duplicados.
# Caso houvesse a necessidade de restringir os dados duplicados seria usada a função "pd.dataframe.drop_duplicates()", passando como parametro o nome da coluns e axis=1
# Então, neste caso específico, não faz sentido remover os "valores duplicados" pois cada apartamento é único.


## Dependências e carregamento do Dataframe

In [ ]:
import pandas as pd
import numpy as np
import statistics as st
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
chennai_df = pd.read_csv('Chennai.csv')
chennai_df['dataset'] = 'Chennai'

hyderabad_df = pd.read_csv('Hyderabad.csv')
hyderabad_df['dataset'] = 'Hydebarad'

kolkata_df = pd.read_csv('Kolkata.csv')
kolkata_df['dataset'] = 'Kolkata'

In [ ]:
#chennai_df.head()

In [ ]:
#hyderabad_df.head()

In [ ]:
#kolkata_df.head()

In [ ]:
df = pd.concat([chennai_df, hyderabad_df, kolkata_df])

### Tratamento Inicial

In [ ]:
# Foi retirada a palavra 'bathrooms' da coluna bathroom para ser possível a contabilização dos banheiros,
# em seguida é retirada as virgulas da coluna price para em seguida converter-la para 'float64' e assim ser possível a realização de calculas e demonstração de gráficos

In [ ]:
df['bathroom'] = df['bathroom'].str.replace('bathrooms', '')

In [ ]:
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype(np.float64)

## Hipóteses

In [164]:
df.corr()['price']

C:\Users\anjos\AppData\Local\Temp\ipykernel_30676\441180871.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['price']


bedroom    0.505890
price      1.000000
area       0.422711
Name: price, dtype: float64

In [ ]:
# Com base no resultado do cálculo do método de Correlação de Pearson, que demosntra a relação entre a coluna escolhida, neste caso preço (price), e as demais colunas é possível 
# observer que o preço do apartamento está altamente relacionado à quantidade de banheiros e em seguida à area do apartamento.

## Tratamento de dados ausentes

In [ ]:
df.info()

In [ ]:
# A estratégia escolhida para tratar dados ausentes foi excluir o registro, pois devido a quentidade de amostras a quantidade de registros excluidos é irrelevante

In [ ]:
df.dropna(inplace=True)

## Análise numérica de estatísticas descritivas

##### Este método retorna valores da media, moda, mediana, desvio padrão e quartis do preço para cada atribudo escolhido da coluna de base

In [ ]:
def analise_preco(coluna):
    global df
    
    dct_retorno = {}    
    s_coluna = df[coluna].value_counts()   
    
    for i in range(len(s_coluna)):
        dct_sts = {}
        index = s_coluna.index[i]
        
        
        preco = df[df[coluna] == index]['price']
        
        media = st.mean(preco)
        moda = st.mode(preco)
        mediana = st.median(preco)
        desvio_padrao = st.stdev(preco)
        quartis = st.quantiles(preco)
                
        dct_sts['MEDIA'] = round(media,2)
        dct_sts['MODA'] = round(moda,2)
        dct_sts['MEDIANA'] = round(mediana,2)
        dct_sts['DESVIO PADRAO'] = round(desvio_padrao,2)
        dct_sts['QUARTIS'] = quartis  
        
        #dct_retorno[index] = (dct_sts,preco) 
        dct_retorno[index] = dct_sts 
        
    
    return dct_retorno

##### Este método retorna valores da media, moda, mediana, desvio padrão e quartis da coluna de base

In [ ]:
def analise_desc_num(coluna):
    global df
    
    dct_retorno = {}    
    s_coluna = df[coluna].value_counts()
      
    media = st.mean(s_coluna)
    moda = st.mode(s_coluna)
    mediana = st.median(s_coluna)
    desvio_padrao = st.stdev(s_coluna)
    quartis = st.quantiles(s_coluna)
    
    dct_retorno['MEDIA'] = media
    dct_retorno['MODA'] = moda
    dct_retorno['MEDIANA'] = mediana
    dct_retorno['DESVIO PADRAO'] = desvio_padrao
    dct_retorno['QUARTIS'] = quartis    
    
    return dct_retorno
    

### Análise individual dos atributos das colunas descritivas

#### Coluna - 'seller_type'

In [ ]:
coluna_analise_seller_type = 'seller_type'

statis = analise_desc_num(coluna_analise_seller_type)


print(statis)
df[coluna_analise_seller_type].value_counts().plot(kind='bar')

#### Coluna - 'layout_type'

In [ ]:
coluna_analise_layout_type = 'layout_type'

statis = analise_desc_num(coluna_analise_layout_type)


print(statis)
df[coluna_analise_layout_type].value_counts().plot(kind='bar')

#### Coluna - 'property_type'

In [ ]:
coluna_analise_property_type = 'property_type'

statis = analise_desc_num(coluna_analise_property_type)


print(statis)
df[coluna_analise_property_type].value_counts().plot(kind='bar')

### Analise do preco por atributo da coluna descritiva

#### Coluna - 'bathroom' x 'price'

In [ ]:
coluna_analise_banheiro_x_preco = 'bathroom'

df.plot(kind='scatter', x='price', y=coluna_analise_banheiro_x_preco)

In [ ]:
df.groupby(coluna_analise_banheiro_x_preco)['price'].mean().plot(kind='line', rot=0, figsize=(30,5))

#### Coluna - 'area' x 'price'

In [ ]:
coluna_analise_area_x_preco = 'area'

df.plot(kind='scatter', x='price', y=coluna_analise_area_x_preco)

In [ ]:
df.groupby(coluna_analise_area_x_preco)['price'].mean().plot(kind='line', rot=0, figsize=(30,5))

#### Coluna - 'seller_type' x 'price'

In [ ]:
coluna_analise_seller_type_x_preco = 'seller_type'

statis = analise_preco(coluna_analise_seller_type_x_preco)
print(statis)


df.plot(kind='scatter', x='price', y=coluna_analise_seller_type_x_preco)


In [ ]:
df.groupby(coluna_analise_seller_type_x_preco)['price'].mean().plot(kind='pie', rot=0, figsize=(15,5))

#### Coluna - 'layout_type' x 'price'

In [ ]:
coluna_analise_layout_type_x_preco = 'layout_type'

statis = analise_preco(coluna_analise_layout_type_x_preco)
print(statis)


df.plot(kind='scatter', x='price', y=coluna_analise_layout_type_x_preco)

In [ ]:
df.groupby(coluna_analise_layout_type_x_preco)['price'].mean().plot(kind='pie', rot=0, figsize=(15,5))

#### Coluna - 'property_type' x 'price'

In [ ]:
coluna_analise_property_type_x_preco = 'property_type'

statis = analise_preco(coluna_analise_property_type_x_preco)
print(statis)


df.plot(kind='scatter', x='price', y=coluna_analise_property_type_x_preco)

In [ ]:
df.groupby(coluna_analise_property_type)['price'].mean().plot(kind='pie', rot=0, figsize=(15,5))

## Algoritmo de associação

In [ ]:
df['property_location'] = df['locality'] + ', ' + df['property_type']

In [ ]:
df['property_location']

In [ ]:
data = list(df['property_location'].apply(lambda x:x.split(",") ))

In [ ]:
encoder = TransactionEncoder()

df_encoded = encoder.fit(data).transform(data)
df_encoded = pd.DataFrame(df_encoded, columns=encoder.columns_)
df_encoded

In [ ]:
df_encoded = df_encoded.replace({False: 0, True: 1})
df_encoded

In [ ]:
df_apriori = apriori(df_encoded, 0.001, use_colnames = True)
df_apriori

In [ ]:
df_ar = association_rules(df_apriori, metric="confidence", min_threshold=0.6)
df_ar
df_ar.sort_values(by='confidence', ascending=False)

#### Indicações de anomalias

Analisando os 5 primeiros registros: 

* Em Iyappanthangal, a probabilidade do tipo de proriedade alugada ser um apartamento é de 48%
* 0.2% dos aluguéis da Índia estão em Topsia
* A confiança indica que 100% de quem aluga em Gariahat, faz o aluguel de um apartamento
* Aluguéis em Telecom Nagar representam 0.23% do total do país, e dentro desse valor, 68% são apartamentos
* O suporte entre tipo de propriedade 'apartamento' e a cidade 'Tangra'
indica 0.5% 

###